In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import math 

from dataclasses import dataclass
from typing import Optional

## Utils

In [2]:
import numpy as np
def assert_allclose(a, b, atol=1e-5, rtol=1e-5):
    a, b = np.array(a), np.array(b)
    return np.testing.assert_allclose(a, b, atol=atol, rtol=rtol)

def allclose(a, b, atol=1e-5, rtol=1e-5):
    a, b = np.array(a), np.array(b)
    return np.allclose(a, b, atol=atol, rtol=rtol)

def compute_thetas(head_dim, device="cpu", base=10000.0):
    theta_numerator = torch.arange(0, head_dim, 2).float()
    thetas = 1.0 / (base ** (theta_numerator / head_dim)).to(device)
    return thetas

def my_compute_thetas(head_dim, device="cpu", base=10000.0):
    """
    theta_i = 10000^{-2(i-1)/dim} for i in [1, 2, ..., dim/2]
    """
    assert head_dim % 2 == 0
    exponents = [-2 * (i-1) / head_dim for i in range(1, head_dim//2 + 1)]
    exponents = torch.tensor(exponents, dtype=torch.float).to(device)
    thetas = base ** exponents
    return thetas

In [3]:
a = compute_thetas(128)
b = my_compute_thetas(128)
print(allclose(a, b))
# assert_allclose(a + 1, b)

True


## Model

In [4]:
def precompute_theta_pos_frequencies(
    head_dim: int, 
    seq_len: int,
    device: str,
    base: float = 10000.0
):
    """
    The formula: theta_i = 10000^{-2(i-1)/dim} for i in [1, 2, ..., dim/2]
    Simplified a little bit: a_i = 10000^{2(i-1)/dim}, theta_i = 1/a

    To rotate a pair, we need to know the position of that pair in the vector embedding (i for i in [0, 1, ..., d/2]), and the position of that vector embedding in the sequence of tokens ([j for j in [0, 1, ..., seq_len]]). 
    p_{i, j} = m*theta. 
    where m = M[j] and theta = Theta[i]
    This function basically precompute all the p_{i, j} (in the complex form). 

    Theta = [θ_0, θ_1, ..., θ_{d/2}]
    M = [0, 1, ..., seq_len - 1]
    So basically to compute all possible p_{i, j} we need to compute outer product of Theta and M.

    """
    assert head_dim % 2 == 0, "Dimension must be divisible by 2"
    
    # Build the theta parameters
    # According to the formula theta_i = 10000^{-2(i-1)/dim} for i in [1, 2, ..., dim/2]
    # Shape (head_dim/2,)
    theta_numerator = torch.arange(0, head_dim, 2).float()
    theta = 1.0 / (base ** (theta_numerator / head_dim)).to(device)
    
    # Construct the positions (the "m" parameters)
    # Shape: (seq_len,)
    m = torch.arange(seq_len).to(device)
    
    # Multiply each theta by each position using outer product. freqs below is like the angle for ratation
    # Shape: m * theta: (seq_len,) outer (head_dim/2,) -> (seq_len, head_dim/2)
    freqs = torch.outer(m, theta).float()
 
    # Compute complex numbers in the polar form: c = R * exp(m * theta) ~ cos(m * theta) + i * sin(m * theta)
    # where R = 1, as follows:
    # (seq_len, head_dim/2) -> (seq_len, head_dim/2)
    freqs_complex = torch.polar(torch.ones_like(freqs), freqs)
    return freqs_complex
    
def apply_rotary_embeddings(
    x: torch.Tensor, 
    freqs_complex: torch.Tensor, 
    device: str
) -> torch.Tensor:
    """
    For a vector of dim D, to apply rotation to it, we need to specify the thetas.
    It's like, according to the paper and my own interpretation, we need to first 
    1. chunk the vector into pairs: 
    [v_0, v_1, ..., v_{d-1}] -> [[v_0, v_1], ..., [v_{d-2}, v_{d-1}]]
    2. rotate pairs by pairs
    -> [rotate([v_0, v_1]), ..., rotate([v_{d-2}, v_{d-1}])]
    3. squeeze all the rotated pairs and concatenate them back to a vector. 
    -> [v_0', v_1', ..., v_{d-1}'] where v_i' is rotated v_i.

    It's like a lazy rotation to me. Instead of rotate the vector as a whole, we rotate partitions of it (in the form of pairs), one by one. 
    The key thing here, to rotate a pair, we need a precomputed angle accociated to it. The following code will solve this.
    Args:
        x: input tensor of shape (B, seq_len, n_heads, head_dim)
        freqs_complex: precomputed theta position frequencies, tensor of shape (seq_len, head_dim/2)
        *note*: When pplying kv_cache for inference, x is a tensor with seq_len = 1. freqs_complex is then a tensor of size (1, head_dim/2). This freqs_complex needs to be extracted from the full precompted freqs complex matrix, at the position that aligns with x. For example, when the position of x is `start_pos`, to retrieve the pairs (m, theta) corresponding to the positions [start_pos, start_pos + 1]:
        freqs_complex = full_freqs_complex[start_pos:start_pos + 1]
        
    Outputs:
        x_out: rotated tensor, of shape (B, seq_len, n_heads, head_dim)
    """
    # (B, seq_len, n_heads, head_dim) -> (B, seq_len, n_heads, head_dim/2, 2)
    x_complex = x.float().reshape(*x.shape[:-1], -1, 2)
    # (B, seq_len, n_heads, head_dim/2, 2) -> (B, seq_len, n_heads, head_dim/2)
    x_complex = torch.view_as_complex(x_complex)
    # (seq_len, head_dim/2) -> (1, seq_len, 1, head_dim/2)
    freqs_complex = freqs_complex.unsqueeze(0).unsqueeze(2)
    # broadcast: (B, seq_len, n_heads, head_dim/2) * (1, seq_len, 1, head_dim/2)
    # -> (B, seq_len, n_heads, head_dim/2)
    x_rotated = x_complex * freqs_complex
    # (B, seq_len, n_heads, head_dim/2) -> (B, seq_len, n_heads, head_dim/2, 2)
    x_out = torch.view_as_real(x_rotated)
    # (B, seq_len, n_heads, head_dim/2, 2) -> (B, seq_len, n_heads, head_dim)
    x_out = x_out.reshape(*x.shape)
    return x_out.type_as(x).to(device)

def repeat_kv(x: torch.Tensor, n_rep: int) -> torch.Tensor:
    batch_size, seq_len, n_kv_heads, head_dim = x.shape
    if n_rep == 1: return x
    return (
        x[:, :, :, None, :]
        .expand(batch_size, seq_len, n_kv_heads, n_rep, head_dim)
        .reshape(batch_size, seq_len, n_kv_heads * n_rep, head_dim)
    )
    

@dataclass
class ModelArgs:
    dim: int = 4096
    n_layers: int = 32
    n_heads: int = 32 # Number of heads for queries. like (1, 4096) -> (1, 32, 128)
    n_kv_heads: Optional[int] = None # Numer of heads for K and V. (?) Is this for grouped query attention?
    vocab_size: int = -1 # This will bet set when we load the tokenizer 
    multiple_of: int = 256
    ffn_dim_multiplier: Optional[float] = None
    norm_eps: float = 1e-5

    # Needed for KV cache
    max_batch_size: int = 32
    max_seq_len: int = 2048

    device: str = "cpu"
    
class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        # The gamma parameter
        self.weight = nn.Parameter(torch.ones(dim))
    
    def _norm(self, x: torch.Tensor):
        # formula: a_i_normed = a_i / RMS(a). RMS(a) = sqrt(1/dim * sum(a^2)) 
        # (B, seq_len, dim) * (B, seq_len, 1) -> (B, seq_len, dim)
        # rsqrt: 1 / sqrt(x)
        a = x.pow(2).mean(-1, keepdim=True)
        return x * torch.rsqrt(a + self.eps)
    
    def forward(self, x: torch.Tensor):
        # (dim) * (B, seq_len, dim) = (B, seq_len, dim)
        return self.weight * self._norm(x.float()).type_as(x)
    
class SelfAttention(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        
        self.args = args
        self.n_kv_heads = args.n_heads if args.n_kv_heads is None else args.n_kv_heads
        self.n_heads_q = args.n_heads
        self.n_rep = self.n_heads_q // self.n_kv_heads # indicate how many times the keys and values should be repeated
        self.head_dim = args.dim // args.n_heads 
        assert self.n_rep * self.n_kv_heads == self.n_heads_q 
        assert self.head_dim * args.n_heads == args.dim
        assert self.n_kv_heads * self.head_dim == args.dim / self.n_rep
        
        self.wq = nn.Linear(args.dim, args.n_heads * self.head_dim, bias=False) ## equivalent to nn.Linear(args.dim, args.dim)
        self.wk = nn.Linear(args.dim, self.n_kv_heads * self.head_dim, bias=False) ## equivalent to nn.Linear(args.dim, args.dim / self.n_rep)
        self.wv = nn.Linear(args.dim, self.n_kv_heads * self.head_dim, bias=False) ## equivalent to nn.Linear(args.dim, args.dim / self.n_rep)
        self.wo = nn.Linear(args.n_heads * self.head_dim, args.dim, bias=False) ## equivalent to nn.Linear(args.dim, args.dim)
        
        self.cache_k = torch.zeros((args.max_batch_size, args.max_seq_len, self.n_kv_heads, self.head_dim))
        self.cache_v = torch.zeros((args.max_batch_size, args.max_seq_len, self.n_kv_heads, self.head_dim))
        
    def forward(
        self,
        x: torch.Tensor, # (B, 1, dim). here seq_len = 1
        start_pos: int,
        freqs_complex: torch.Tensor # (1, head_dim/2). this is extracted from the full freqs_complex
    ):
        batch_size, seq_len, _ = x.shape # (B, 1, dim)
        assert seq_len == 1
        xq = self.wq(x) # (B, 1, dim) -> (B, 1, H_Q, head_dim)
        xk = self.wk(x) # (B, 1, dim) -> (B, 1, H_KV, head_dim)
        xv = self.wv(x) # (B, 1, dim) -> (B, 1, H_KV, head_dim)
        
        xq = xq.view(batch_size, seq_len, self.n_heads_q, self.head_dim)
        xk = xk.view(batch_size, seq_len, self.n_kv_heads, self.head_dim)
        xv = xv.view(batch_size, seq_len, self.n_kv_heads, self.head_dim)
        
        ## Apply rotary embedding to q and k only
        xq = apply_rotary_embeddings(xq, freqs_complex, device=x.device)
        xk = apply_rotary_embeddings(xk, freqs_complex, device=x.device)
        
        ## Update the KV cache
        self.cache_k[:batch_size, start_pos:start_pos+seq_len] = xk # (B, 1, H_KV, head_dim)
        self.cache_v[:batch_size, start_pos:start_pos+seq_len] = xv
        
        ## Extract neccessary cached keys and values: all upto the current just computed key and value
        keys = self.cache_k[:batch_size, :start_pos+seq_len]
        values = self.cache_v[:batch_size, :start_pos+seq_len]
        
        ## Repeat keys and values to match the size of queries. Make this standard multihead attention.
        keys = repeat_kv(keys, self.n_rep) # (B, seq_len_KV, H_KV, head_dim) --> (B, seq_len_KV, H_Q, head_dim)
        values = repeat_kv(values, self.n_rep) # (B, seq_len_KV, H_KV, head_dim) --> (B, seq_len_KV, H_Q, head_dim)
        
        ## Transpose for matmul compability
        xq = xq.transpose(1, 2) # (B, 1, H_Q, head_dim) -> (B, H_Q, 1, head_dim)
        keys = keys.transpose(1, 2) # (B, seq_len_KV, H_Q, head_dim) -> (B, H_Q, seq_len_KV, head_dim)
        values = values.transpose(1, 2) # (B, seq_len_KV, H_Q, head_dim) -> (B, H_Q, seq_len_KV, head_dim)
        
        # (B, H_Q, 1, head_dim) @ (B, H_Q, head_dim, seq_len_KV) -> (B, H_Q, 1, seq_len_KV)
        scores = torch.matmul(xq, keys.transpose(2, 3)) / math.sqrt(self.head_dim)
        # (B, H_Q, 1, seq_len_KV) -> (B, H_Q, 1, seq_len_KV)
        scores = F.softmax(scores.float(), dim=-1).type_as(xq)
        # (B, H_Q, 1, seq_len_KV) @ (B, H_Q, seq_len_KV, head_dim) -> (B, H_Q, 1, head_dim)
        output = torch.matmul(scores, values)
        ## Concat output heads
        assert self.n_heads_q * self.head_dim == self.args.dim
        # (B, H_Q, 1, head_dim) -> (B, 1, dim)
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_len, -1) # last dim: self.n_heads_q * self.head_dim
        assert output.shape[-1] == self.n_heads_q * self.head_dim
        output = self.wo(output) # (B, 1, dim) -> (B, 1, dim)
        return output
    
class FeedForward(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()

        hidden_dim = 4 * args.dim
        hidden_dim = int(2 * hidden_dim / 3)
        if args.ffn_dim_multiplier is not None:
            hidden_dim = int(args.ffn_dim_multiplier * hidden_dim)
        # Round the hidden_dim to the nearest multiple of the multiple_of parameter
        hidden_dim = args.multiple_of * ((hidden_dim + args.multiple_of - 1) // args.multiple_of)

        self.w1 = nn.Linear(args.dim, hidden_dim, bias=False)
        self.w2 = nn.Linear(hidden_dim, args.dim, bias=False)
        self.w3 = nn.Linear(args.dim, hidden_dim, bias=False)

    def forward(self, x: torch.Tensor):
        """
        shape transformation: dim -> hidden_dim -> dim.
        Vanilla: FFN(x) = ReLU(xW1 + b1)W2 + b2 
        SwiGLU: FFN(x) = (Swish(xW1) ⊙ xW3)W2
        """
        swish = F.silu(self.w1(x)) # (B, Seq_Len, Dim) --> (B, Seq_Len, Hidden_Dim)
        x_V = self.w3(x) # (B, Seq_Len, Dim) --> (B, Seq_Len, Hidden_Dim)
        x = swish * x_V # (B, Seq_Len, Hidden_Dim) * (B, Seq_Len, Hidden_Dim) --> (B, Seq_Len, Hidden_Dim)
        x = self.w2(x) # (B, Seq_Len, Hidden_Dim) --> (B, Seq_Len, Dim)
        return x
        
    
class EncoderBlock(nn.Module):
    def __init__(self, args: ModelArgs):
        super().__init__()
        self.n_heads = args.n_heads 
        self.dim = args.dim 
        self.head_dim = args.dim // args.n_heads 
        
        self.attention = SelfAttention(args)
        self.feed_forward = FeedForward(args)
        
        # Normalize BEFORE the attention block
        self.attention_norm = RMSNorm(args.dim, eps=args.norm_eps)
        # Normalize BEFORE the feedforward block
        self.ffn_norm = RMSNorm(args.dim, eps=args.norm_eps)
        
    def forward(self, x: torch.Tensor, start_pos: int, freqs_complex: torch.Tensor):
        # (B, seq_len, dim) + (B, seq_len, dim) -> (B, seq_len, dim)
        h = x + self.attention.forward(
            self.attention_norm(x), start_pos, freqs_complex
        )
        # (B, seq_len, dim) + (B, seq_len, dim) -> (B, seq_len, dim)
        out = h + self.feed_forward.forward(self.ffn_norm(h))
        return out
    
class Transformer(nn.Module):
    def __init__(self, args: ModelArgs) -> None:
        super().__init__()
        assert args.vocab_size != -1, "Vocab size must be set"
        self.args = args
        self.vocab_size = args.vocab_size
        self.n_layers = args.n_layers

        self.tok_embeddings = nn.Embedding(self.vocab_size, args.dim)
        self.layers = nn.ModuleList(
            [EncoderBlock(args) for _ in range(args.n_layers)]
        )
        self.norm = RMSNorm(args.dim, eps=args.norm_eps)
        self.output = nn.Linear(args.dim, args.vocab_size, bias=False)

        self.freqs_complex = precompute_theta_pos_frequencies(
            self.args.dim // self.args.n_heads,
            self.args.max_seq_len * 2,
            device=self.args.device
        )

    def forward(self, tokens: torch.Tensor, start_pos: int):
        """
        Here we use KV cache, therefore only one token is needed to compute the next token.
        """
        batch_size, seq_len = tokens.shape
        assert seq_len == 1, "Only 1 token"

        # (B, seq_len) -> (B, seq_len, dim)
        h = self.tok_embeddings(tokens)

        # Retrieve te pairs (m, theta) corresponding to the positions [start_pos, start_pos + seq_len]
        freqs_complex = self.freqs_complex[start_pos:start_pos + seq_len]

        for layer in self.layers:
            h = layer(h, start_pos, freqs_complex)
        h = self.norm(h)
        output = self.output(h).float()
        return output

## Inference

In [5]:
from typing import Optional
import torch
import time
from pathlib import Path
import json
from sentencepiece import SentencePieceProcessor
from tqdm import tqdm

In [6]:
class Llama:
    def __init__(
        self, 
        model: Transformer, 
        tokenizer: SentencePieceProcessor, 
        model_args: ModelArgs
    ):
        self.model = model
        self.tokenizer = tokenizer
        self.args = model_args
    
    @staticmethod
    def build(
        checkpoints_dir: str,
        tokenizer_path: str,
        load_model: bool,
        max_seq_len: int,
        max_batch_size: int,
        device: str
    ):
        prev_time = time.time()
        if load_model:
            checkpoints = sorted(Path(checkpoints_dir).glob('*.pth'))
            assert len(checkpoints) > 0, "No checkpoints files found"
            chk_path = checkpoints[0]
            print(f"Loading checkpoint {chk_path}")
            checkpoint = torch.load(chk_path, map_location="cpu")
            print(f"Loaded checkpoint in {(time.time() - prev_time):.2f}s")
            prev_time = time.time()
        with open(Path(checkpoints_dir) / "params.json", "r") as f:
            params = json.loads(f.read())
        
        model_args = ModelArgs(
            max_seq_len=max_seq_len,
            max_batch_size=max_batch_size,
            device=device,
            **params
        )
        
        tokenizer = SentencePieceProcessor()
        tokenizer.load(tokenizer_path)
        model_args.vocab_size = tokenizer.vocab_size()
        
        if device == "cuda":
            torch.set_default_tensor_type(torch.cuda.HalfTensor)
        elif device == "cpu":
            torch.set_default_tensor_type(torch.BFloat16Tensor)
        else:
            raise NotImplementedError()
        model = Transformer(model_args).to(device)
        
        if load_model:
            del checkpoint['rope.freqs']
            model.load_state_dict(checkpoint, strict=True)
            print(f"Loaded state dict in {time.time() - prev_time:.2f}s")
        
        return Llama(model, tokenizer, model_args)
    
    def text_completion(self, prompts: list[str], temperature: float = 0.6, top_p: float = 0.9, max_gen_len: Optional[int] = None):
        if max_gen_len is None:
            max_gen_len = self.args.max_seq_len - 1
        # Convert each prompt into tokens
        prompt_tokens = [self.tokenizer.encode(prompt, out_type=int, add_bos=True, add_eos=False) for prompt in prompts]
        # Make sure the batch size is not too large
        batch_size = len(prompt_tokens)
        assert batch_size <= self.args.max_batch_size, f"batch size must be less than or equal to {self.args.max_batch_size}"
        max_prompt_len = max(len(prompt) for prompt in prompt_tokens)
        # Make sure the prompt length is not larger than the maximum sequence length
        assert max_prompt_len <= self.args.max_seq_len, f"prompt length must be less than or equal to {self.args.max_seq_len}"
        total_len = min(self.args.max_seq_len, max_gen_len + max_prompt_len)

        # Create the list that will contain the generated tokens, along with the initial prompt tokens
        pad_id = self.tokenizer.pad_id()
        tokens = torch.full((batch_size, total_len), pad_id, dtype=torch.long, device=device)
        for k, t in enumerate(prompt_tokens):
            # Populate the initial tokens with the prompt tokens
            tokens[k, : len(t)] = torch.tensor(t, dtype=torch.long, device=device)
        
        eos_reached = torch.tensor([False] * batch_size, device=device)
        prompt_tokens_mask = tokens != pad_id # True if the token is a prompt token, False otherwise
        cur_iterator = tqdm(range(1, total_len), desc="Generating tokens")
        for cur_pos in cur_iterator:
            with torch.no_grad():
                logits = self.model.forward(tokens[:, cur_pos-1:cur_pos], cur_pos)
            if temperature > 0:
                # The temperature is applied before the softmax
                probs = torch.softmax(logits[:, -1] / temperature, dim=-1)
                next_token = self._sample_top_p(probs, top_p)
            else:
                # Greedily select the token with the max probability
                next_token = torch.argmax(logits[:, -1], dim=-1)

            next_token = next_token.reshape(-1)
            # Only replace token if it is a padding token
            next_token = torch.where(prompt_tokens_mask[:, cur_pos], tokens[:, cur_pos], next_token)
            tokens[:, cur_pos] = next_token
            # EOS is reached only if we found an EOS token for a padding position
            eos_reached |= (~prompt_tokens_mask[:, cur_pos]) & (next_token == self.tokenizer.eos_id)
            if all(eos_reached):
                break

        out_tokens = []
        out_text = []
        for prompt_index, current_prompt_tokens in enumerate(tokens.tolist()):
            # Cut to the EOS token, if present
            if self.tokenizer.eos_id in current_prompt_tokens:
                eos_idx = current_prompt_tokens.index(self.tokenizer.eos_id)
                current_prompt_tokens = current_prompt_tokens[:eos_idx]
            out_tokens.append(current_prompt_tokens)
            out_text.append(self.tokenizer.decode(current_prompt_tokens))
        return (out_tokens, out_text)
    
    def _sample_top_p(self, probs, p):
        # (B, vocab_size)
        probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)
        # (B, vocab_size)
        probs_sum = torch.cumsum(probs_sort, dim=-1)
        # (B, vocab_size)
        # (Substracting "probs_sort" shifts the cumulative sum by 1 position to the right before masking)
        mask = probs_sum - probs_sort > p 
        # Zero out all the probabilities of tokens that are not selected by the Top P
        probs_sort[mask] = 0.0 
        # Redistribute the probabilities so that they sum up to 1.
        probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))
        # Sample a token (its index) from the top p distribution
        next_token = torch.multinomial(probs_sort, num_samples=1)
        # Get the token position in the vocabulary corresponding to the sampled index
        next_token = torch.gather(probs_idx, -1, next_token) 
        return next_token

In [7]:
model_path = '/kaggle/input/llama-2/pytorch/7b/1/'
tokenizer_path = '/kaggle/input/llama-2/pytorch/7b/1/tokenizer.model'
torch.manual_seed(0)

allow_cuda = False
device = 'cuda' if torch.cuda.is_available() and allow_cuda else 'cpu'
model = Llama.build(
    checkpoints_dir=model_path,
    tokenizer_path=tokenizer_path,
    load_model=True,
    max_seq_len=1024,
    max_batch_size=4,
    device=device
)

Loading checkpoint /kaggle/input/llama-2/pytorch/7b/1/consolidated.00.pth
Loaded checkpoint in 123.01s
Loaded state dict in 98.77s


In [8]:
prompts = [
    "Simply put, the theory of relativity states that ",
    "If Google was an Italian company founded in Milan, it would",
    # Few shot promt
    """Translate English to French:

sea otter => loutre de mer
peppermint => menthe poivrée
plush girafe => girafe peluche
cheese =>""",
    # Zero shot prompt
    """Tell me if the following person is actually Doraemon disguised as human:
Name: Umar Jamil
Decision: 
    """
]
out_tokens, out_texts = (model.text_completion(prompts, max_gen_len=64))
assert len(out_texts) == len(prompts)
for i in range(len(out_texts)):
    print(f'{out_texts[i]}')
    print('-' * 50)

Generating tokens: 100%|██████████| 110/110 [42:24<00:00, 23.14s/it]

Simply put, the theory of relativity states that 1) time is relative to the observer, 2) mass is relative to the observer, 3) speed is relative to the observer, and 4) energy is relative to the observer.ergy is relative to the observer.
In 1905, Albert Einstein published a paper called "On the Electrodynamics of Moving Bodies" in which he proposed that the speed of light is a constant for all observers, and that the laws of
--------------------------------------------------
If Google was an Italian company founded in Milan, it would be the 4th largest company in Italy. duh!
Google is a global company. Google has its headquarters in California, but its employees are all over the world. Google has offices in 38 countries.
If Google was an Italian company founded in Milan, it would be the 4th largest company in Italy.
Google is a global company. Google has its headquarters in California, but its employees are all over the world. Google has offices in 38
-----------------------------------

## Misc